# Comments classification: offensive/non-offensive

1. Import Dataset

In [2]:
import pandas as pd
df = pd.read_csv('Dataset/CVMAD_04.csv', encoding='UTF-8')### if it doesn't work try 'utf-8-sig'
df.head()

,content,decision
0,یخ علی الخماح مخسر للسیده وجها مازال کتهضرشوهت...,1
1,یاک مطلقها علاش کیحاسبها داب,1
2,یاخوفي من بدعة التفاخر بالقتل والتشرمیل والمعص...,1
3,يوم يكون الصوت سعره يتحدى 30درهم هنا سيصبح الت...,0
4,يوسف زروالي هو لقتقصدك لدخلو معاه جميعة شباب ا...,0


In [ ]:
import seaborn as sns # visulization
sns.countplot(x=df['decision'])

2. Data pre-processing

In [3]:
import re
def removeWeirdChars(text):
    weirdPatterns = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               u"\u2069"
                               u"\u2066"
                               u"\u200c"
                               u"\u2068"
                               u"\u2067"
                               "]+", flags=re.UNICODE)
    return weirdPatterns.sub(r'', text)

In [4]:
df.content = [removeWeirdChars(d) for d in df.content]

In [4]:
df.content[0:10]

0    یخ علی الخماح مخسر للسیده وجها مازال کتهضرشوهت...
1                         یاک مطلقها علاش کیحاسبها داب
2    یاخوفي من بدعة التفاخر بالقتل والتشرمیل والمعص...
3    يوم يكون الصوت سعره يتحدى 30درهم هنا سيصبح الت...
4    يوسف زروالي هو لقتقصدك لدخلو معاه جميعة شباب ا...
5                                             يوسف رجل
6        يوسف الزروالي هو لي دخلو لي جمعية شباب الملكي
7                        يوزع شكون فيق سكيزو من السبات
8      يوتيب تسبب ليا بمرض نفسي وليت كنخاف ومرضت نفسيا
9    يهدر واش تتوقع منو وجاية ملمريخ نتا واش قودك ي...
Name: content, dtype: object

3. Data Encoding

Word Embedding

In [5]:
import numpy as np
word_embeddings = {}
f = open('ma_model_cbow_mix.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
print('Total %s word vectors in MA CBOW model 300d.' % len(word_embeddings))

Total 82906 word vectors in MA CBOW model 300d.


In [6]:
def get_sentence_vectors(sentence):
    sentence_vector = np.zeros((300,))
    if len(sentence)==0:
        return sentence_vector
    for word in sentence:
        if word in word_embeddings:
            sentence_vector += word_embeddings[word]
    sentence_vector = sentence_vector/len(sentence)
    return sentence_vector

In [8]:
len(word_embeddings)

82906

In [7]:
word_embeddings['علاش']

array([-0.04573694,  0.10975436, -0.54744524,  0.10996349,  0.20633675,
        0.90674895,  0.11491557,  0.72445756,  0.58139473, -0.2946483 ,
       -0.7712647 , -0.17335273,  0.33888417, -0.20815212,  0.46175632,
        0.38756797,  0.28463918, -0.03309639, -0.08293769, -0.13326049,
       -0.58587915, -0.48430493, -0.16314602, -0.38404787,  0.08127248,
       -0.8705088 ,  0.33157548,  0.3544283 ,  0.5581337 ,  0.08885915,
       -1.1427025 ,  0.92506343, -0.09592154,  0.5934387 ,  0.3629863 ,
       -0.21132165,  0.30229187,  0.16288331, -0.31264427, -0.04881828,
       -0.08142379, -1.4429231 , -0.66444516, -1.4426838 , -1.0821352 ,
       -0.05486058,  0.8075012 ,  0.6802713 ,  1.0687793 , -1.7807748 ,
       -1.0383372 ,  1.5321866 , -0.8323125 ,  0.3871929 , -0.6774203 ,
        1.2722178 ,  0.6946758 , -0.8461339 ,  1.0099423 ,  0.37826127,
        0.580605  , -0.18887214, -0.8269939 ,  0.958509  ,  0.4126865 ,
       -2.1811097 ,  1.2467979 , -0.53286076, -1.0175996 , -1.90

In [8]:
df["content"] = df["content"].apply(get_sentence_vectors)
df.head()

,content,decision
0,"[0.03822386347585254, 0.030719361233490484, -0...",1
1,"[-0.016081052499690225, -0.011788641102612019,...",1
2,"[0.014637194904140555, 0.01521033041895582, -0...",1
3,"[0.027888019090971432, 0.02286620584929043, -0...",0
4,"[0.04122190000306742, 0.028468514847404817, -0...",0


4. Modeling

5. Data splitting

In [9]:
X = df.content
y = df.decision

In [10]:
X[0:3]

0    [0.03822386347585254, 0.030719361233490484, -0...
1    [-0.016081052499690225, -0.011788641102612019,...
2    [0.014637194904140555, 0.01521033041895582, -0...
Name: content, dtype: object

In [11]:
X.shape

(23000,)

In [13]:
y[0:3]

0    1
1    1
2    1
Name: decision, dtype: int64

In [12]:
""" Sckaling and Splitting the dataset into training, validation and test sets """
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#scaler = StandardScaler()
#X_sacaled=scaler.fit_transform(X)

train_X, test_X, train_y, test_y = train_test_split(X.to_list(), y.to_list(), random_state=50, test_size=0.3)

In [13]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(train_X, train_y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
pred_y=clf.predict(test_X)

In [15]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(test_y, pred_y)
cnf_matrix

array([[1171, 2254],
       [1085, 2390]], dtype=int64)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.52      0.34      0.41      3425
           1       0.51      0.69      0.59      3475

   micro avg       0.52      0.52      0.52      6900
   macro avg       0.52      0.51      0.50      6900
weighted avg       0.52      0.52      0.50      6900

